In [1]:
# Cell 0

network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [2]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [3]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [4]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)
print(icon_service.get_balance(wallet.get_address())/10**18)
print(icon_service.get_balance(btest_wallet.get_address())/10**18)

1751.1839507896948
4239.9559852


In [5]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
1751.1839507896948


In [6]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
4239.9559852


In [7]:
user1 = KeyWallet.load("keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

2436.4288446
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
2390.4790276
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [117]:
# Cell 5
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa0f715fb2c4bc8f4c6399c2cc26167a27be0aa61'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxbabed822d59b605dbeb6322735c529b292baac3b'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx1379084f45776301abda3849c6e374f460ee0155'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx71dda2221bf88faddc8f84b72ffd6db296e5609e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxfd09787f23d23b945fa0c7eb55b5aa69425da1c8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx27aa3bf62145822e60d85fa5d18dabdcff5b9ada'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx01eee12b6614e5328e0a84261652cb7f055e0176'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd7b3e71dcff3d75392216e208f28ef68e8a54ec0'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx799f724e02560a762b5f2bd3b6d2d8d59d7aecc1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx4d0768508a7ff550de4405f27aebfb8831565c19'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx663f9d59163846d9f6c6f7b586858c59aa8878a9'}}

In [118]:
# Cell 6
# Define deploy and send_tx functions

def compress():
    """
    Compress all SCORE folders in the core_contracts and toekn_contracts folders
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res.get('scoreAddress', '')

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet):
    """
    Compress, Deploy and Configure all SCOREs
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')

    governance = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    contracts['governance']['SCORE'] = governance
    for score in deploy:
        contracts[score]['SCORE'] = deploy_SCORE(contracts[score], {'_governance': governance}, wallet, 0)
    contracts['staking']['SCORE'] = deploy_SCORE(contracts['staking'], {}, wallet, 0)
    contracts['sicx']['SCORE'] = deploy_SCORE(contracts['sicx'], {'_admin': contracts['staking']['SCORE']}, wallet, 0)

    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}

    txns = [{'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}},
            {'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}},
            {'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}}]

    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    print(result)
    return result


In [119]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

results = {}
deploy_all(btest_wallet)
print('------------------------------------------------------------------------------------------------------------------')
print(contracts)
print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
print(get_scores_json(contracts))

core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1

core_contracts/loans.zip


...

Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxb86416a133678051c47693f7c038e3772754b6c0', 'indexed': ['AssetAdded(Address,str,bool)', 'cx8f13e0c771b436a83754d1e2d60d678863624bcf', 'sICX', '0x1'], 'data': []} 

{'scoreAddress': 'cxb86416a133678051c47693f7c038e3772754b6c0', 'indexed': ['AssetAdded(Address,str,bool)', 'cxb800e009cb966c5bebdfc7a47199038f4cb41567', 'bnUSD', '0x0'], 'data': []} 

{'scoreAddress': 'cxb86416a133678051c47693f7c038e3772754b6c0', 'indexed': ['AssetAdded(Address,str,bool)', 'cxcc57e12685c65f75f304e9d8655d2d3aa0212e20', 'BALN', '0x1'], 'data': []} 

{'scoreAddress': 'cxb86416a133678051c47693f7c038e3772754b6c0', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']} 

------------------------------------------------------------------------------------------------------------------
{'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': 'cxb86416

In [226]:
# Cell 8
# Deploy or Update a single SCORE

compress()
update = 1
contract = contracts['rewards']
params = {}
# params = {'_governance': contracts['governance']['SCORE']}
deploy_SCORE(contract, params, btest_wallet, update)


core_contracts/rewards.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1



'cx1013402961b972e30107a345f403e6caf2c4eb30'

In [235]:

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_pid':2 , '_value':10 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(res)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
{'txHash': '0x9b0a93d4d61d7a7e50889fda5b9f010ae20d6b47945f1adcf8ddb3628e93929c', 'blockHeight': 14972586, 'blockHash': '0xeffdbbfd58a7859e41d3189bdbd8a3e9b17e8fc33a744ebf7af129c903a603d4', 'txIndex': 1, 'to': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'stepUsed': 292320, 'stepPrice': 10000000000, 'cumulativeStepUsed': 292320, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [185]:
# Cell 10

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method('getRecipientsSplit')\
                    .params({}) \
                    .build()
result = icon_service.call(call)
print(result)

{'Worker Tokens': '0x2c68af0bb140000', 'Reserve Fund': '0xb1a2bc2ec50000', 'DAOfund': '0x31f5c4ed2768000', 'Loans': '0x3782dace9d90000', 'SICXICX': '0x16345785d8a0000', 'SICXbnUSD': '0x26db992a3b18000', 'OMMsICX': '0x0', 'OMMIUSDC': '0x0', 'OMMUSDB': '0x0'}


In [120]:
contracts['dex']['SCORE']

'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a'

In [97]:
# Cell 9
# transfer icx to dex

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 0
Failure: {'code': 1, 'message': 'integer division or modulo by zero'}


In [12]:
# Cell 10

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

0.0

# Creating OMM/sICX LP, OMM/IUSDC

In [229]:
# Cell 18a
# adding Second currency IUSDC in dex
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("dexAddQuoteCoin")\
    .params({'_address':'cx65f639254090820361da483df233f6d0e69af9b7'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1


In [230]:
# Cell 18a
# adding Second currency USDB in dex
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("dexAddQuoteCoin")\
    .params({'_address':'cxaa068556df80f9917ef146e889f0b2c4b13ab634'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1


In [231]:
# Cell 11
# transfer sicx to the wallet

params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(100 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("stakeICX")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx8f13e0c771b436a83754d1e2d60d678863624bcf', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x56bc75e2d63100000', '0x4e6f6e65']} 

{'scoreAddress': 'cx8f13e0c771b436a83754d1e2d60d678863624bcf', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x56bc75e2d63100000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxdaf57b8cf5921936804e83667241b6865ce962e8', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x56bc75e2d63100000'], 'data': ['100.0 sICX minted to hxe7af5fcfd8dfc67530a01a0e403882687528dfcb']} 



In [232]:
# Cell 12
# check sicx balance

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['sicx']['SCORE'])\
                    .method('balanceOf')\
                    .params({'_owner': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

170.0

In [125]:
# Cell 13
# gives pools name

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

['SICXICX']


In [234]:
# Cell 14
# deposit sicx to the dex to create the pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(res)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
{'txHash': '0x8b2ef67679d70d2c4bcdb585e2991499148ebf7d6c01349e9fd3294e258c8502', 'blockHeight': 14972526, 'blockHash': '0x3499d221aede62ded062ade29be970abafbb37cbdc8a06198d4b0c5621fb9a1c', 'txIndex': 1, 'to': 'cx8f13e0c771b436a83754d1e2d60d678863624bcf', 'stepUsed': 455440, 'stepPrice': 10000000000, 'cumulativeStepUsed': 455440, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [138]:
# Cell 15
# get OMM balance for the wallet

call = CallBuilder().from_(wallet.get_address())\
                    .to('cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6')\
                    .method('balanceOf')\
                    .params({'_owner': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

9600.0

In [189]:
# Cell 16a
# deposit omm to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to('cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cxcc57e12685c65f75f304e9d8655d2d3aa0212e20', 'indexed': ['Mint(Address,int,bytes)', 'cx1013402961b972e30107a345f403e6caf2c4eb30'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cxcc57e12685c65f75f304e9d8655d2d3aa0212e20', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx1013402961b972e30107a345f403e6caf2c4eb30', '0x152d02c7e14af6800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxcc57e12685c65f75f304e9d8655d2d3aa0212e20', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx1013402961b972e30107a345f403e6caf2c4eb30', 'cx649d7dd85dda8efd94a8b0b124ba7e8c566d63b0', '0x43c33c1937564800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxcc57e12685c65f75f304e9d8655d2d3aa0212e20', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx1013402961b972e30107a345f403e6caf2c4eb30', 'cxa2fd8f7d883ed35aa9d14ceb3da2c3d3e0cf5e37', '0x10f0cf064dd59200000'], 'data': ['0x4e6

In [191]:
# Cell 16b
# deposit iusdc to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * 10**6, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to('cx65f639254090820361da483df233f6d0e69af9b7')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)

print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 0
Failure: {'code': 32, 'message': "USDICX, BandChain, cx7171e2f5653c1b9c000e24228276b8d24e84f10d, Exception: 'NoneType' object has no attribute '_IconScoreBase__is_func_readonly'"}


In [192]:
# Cell 16c
# deposit usdb to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to('cxaa068556df80f9917ef146e889f0b2c4b13ab634')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)

print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 0
Failure: {'code': 32, 'message': "USDICX, BandChain, cx7171e2f5653c1b9c000e24228276b8d24e84f10d, Exception: 'NoneType' object has no attribute '_IconScoreBase__is_func_readonly'"}


In [162]:

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_pid':2 , '_value':10 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 0
Failure: {'code': 32, 'message': "USDICX, BandChain, cx7171e2f5653c1b9c000e24228276b8d24e84f10d, Exception: 'NoneType' object has no attribute '_IconScoreBase__is_func_readonly'"}


In [131]:
# Cell 17a
# add base and quote token addresses for the paris that we want to create a pool of omm/sicx.

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken':'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6' , '_quoteToken': contracts['sicx']['SCORE'], '_maxBaseValue': 10 * ICX, '_quoteValue': 30990197728645406}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['MarketAdded(int,Address,Address,int,int)', '0x2', 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6', 'cx8f13e0c771b436a83754d1e2d60d678863624bcf'], 'data': ['0x8ac7230489e80000', '0x6e196baf22911e']} 

{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['Add(int,Address,int)', '0x2', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x56bc75e2d63100000']} 

{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['TransferSingle(Address,Address,Address,int,int)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cxf000000000000000000000000000000000000000', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x2', '0x56bc75e2d63100000']} 



In [132]:
# Cell 17b
# add base and quote token addresses for the paris that we want to create a pool of omm/iusdc.

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken':'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6' , '_quoteToken': 'cx65f639254090820361da483df233f6d0e69af9b7', '_maxBaseValue': 10 * ICX, '_quoteValue': 10 * 10**6}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['MarketAdded(int,Address,Address,int,int)', '0x3', 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6', 'cx65f639254090820361da483df233f6d0e69af9b7'], 'data': ['0x8ac7230489e80000', '0x989680']} 

{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['Add(int,Address,int)', '0x3', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x56bc75e2d63100000']} 

{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['TransferSingle(Address,Address,Address,int,int)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cxf000000000000000000000000000000000000000', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x3', '0x56bc75e2d63100000']} 



In [135]:
# Cell 17c
# add base and quote token addresses for the paris that we want to create a pool of omm/usdb.

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken':'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6' , '_quoteToken': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', '_maxBaseValue': 10 * ICX, '_quoteValue': 10 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['MarketAdded(int,Address,Address,int,int)', '0x4', 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6', 'cxaa068556df80f9917ef146e889f0b2c4b13ab634'], 'data': ['0x8ac7230489e80000', '0x8ac7230489e80000']} 

{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['Add(int,Address,int)', '0x4', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x56bc75e2d63100000']} 

{'scoreAddress': 'cxa2dee3c217cb83d2f5ab885b8b95762cd3bef75a', 'indexed': ['TransferSingle(Address,Address,Address,int,int)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cxf000000000000000000000000000000000000000', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x4', '0x56bc75e2d63100000']} 



In [133]:
# Cell 18a
# setting market name for ommsICX pair

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':2, '_name': 'OMMsICX'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1


['SICXICX', 'OMMsICX']


In [134]:
# Cell 18b
# setting market name for omm/iusdc pair

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':2, '_name': 'OMMIUSDC'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1


['SICXICX', 'OMMsICX', 'OMMIUSDC']


In [136]:
# Cell 18bc
# setting market name for omm/usdb pair

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':2, '_name': 'OMMUSDB'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Status: 1


['SICXICX', 'OMMsICX', 'OMMIUSDC', 'OMMUSDB']


In [221]:
# Cell 33

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'OMMUSDB', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

{'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb': '0x5525fc05359cb2aa'}
